In [ ]:
!pip install gdown

In [ ]:
!gdown --id 1x8VdZBs25F9EkWnJw1XR4xnUfBHn0eyY

In [ ]:
!gdown --id 1by9yXKye9QkaN9dAqWnMJkNglG9AD3rM

In [ ]:
!pip install rasterio pillow tqdm

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import rasterio
from rasterio.windows import Window
from rasterio.plot import reshape_as_image
from rasterio.warp import transform_bounds
from rasterio import features

import geopandas as gpd
from shapely.geometry import shape

import folium


In [ ]:
"""
Splits a GeoTIFF into 1024*1024 tiles.
Preserves geotransform, CRS, and metadata.
"""

tif_path = "/content/Hanumannagar_Postflood_Orthomosaic.tif"
out_dir = "tiles_postflood"
os.makedirs(out_dir, exist_ok=True)

tile_size = 1024


# Open TIFF and create tiles

with rasterio.open(tif_path) as src:
    width, height = src.width, src.height
    profile = src.profile

    x_steps = list(range(0, width, tile_size))
    y_steps = list(range(0, height, tile_size))

    total_tiles = len(x_steps) * len(y_steps)

    for i in tqdm(x_steps, desc="Tiling columns"):
        for j in y_steps:
            w = min(tile_size, width - i)
            h = min(tile_size, height - j)
            window = Window(i, j, w, h)

            tile = src.read(window=window)

            tile_profile = profile.copy()
            tile_profile.update({
                "width": w,
                "height": h,
                "transform": rasterio.windows.transform(window, src.transform)
            })

            out_path = os.path.join(out_dir, f"tile_{i}_{j}.tif")
            with rasterio.open(out_path, "w", **tile_profile) as dst:
                dst.write(tile)

print(f"Tiling complete! Tiles saved in {out_dir}")

Tiling columns: 100%|██████████| 69/69 [09:27<00:00,  8.22s/it]

Tiling complete! Tiles saved in tiles_postflood


In [ ]:
print(f"TIFF size: {width} x {height}, Bands: {src.count}")

TIFF size: 60971 x 88127, Bands: 4


In [ ]:
"""
Generate NDWI-based water masks for all pre-flood and post-flood satellite tiles.

NDWI (Normalized Difference Water Index):
    NDWI = (Green - NIR) / (Green + NIR)

Pixel classification:
    Water pixel → NDWI > threshold
    Land pixel  → NDWI <= threshold


1. Reads each tile from pre-flood and post-flood folders.
2. Computes NDWI using Green (Band 2) and NIR (Band 4).
3. Converts NDWI to a binary mask (1 = water, 0 = land).
4. Saves the mask as a GeoTIFF while preserving georeferencing.
"""

preflood_tile_dir = "tiles_Preflood"
postflood_tile_dir = "tiles_postflood"

preflood_mask_dir = "masks_preflood"
postflood_mask_dir = "masks_postflood"

os.makedirs(preflood_mask_dir, exist_ok=True)
os.makedirs(postflood_mask_dir, exist_ok=True)

ndwi_threshold = 0.1 


# Function to compute NDWI mask and save as TIFF

def compute_ndwi_mask_tile(tile_path, out_dir, threshold=0.1):
    with rasterio.open(tile_path) as src:
        green = src.read(2).astype(float)  # Band 2 = Green
        nir = src.read(4).astype(float)    # Band 4 = NIR

        ndwi = (green - nir) / (green + nir + 1e-6)
        mask = (ndwi > threshold).astype(np.uint8)

        profile = src.profile
        profile.update(dtype=rasterio.uint8, count=1)  # single band mask

        out_path = os.path.join(out_dir, os.path.basename(tile_path))
        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(mask, 1)


# Process all pre-flood tiles

preflood_tiles = os.listdir(preflood_tile_dir)
for tile_file in tqdm(preflood_tiles, desc="Processing pre-flood tiles"):
    tile_path = os.path.join(preflood_tile_dir, tile_file)
    compute_ndwi_mask_tile(tile_path, preflood_mask_dir, ndwi_threshold)


# Process all post-flood tiles

postflood_tiles = os.listdir(postflood_tile_dir)
for tile_file in tqdm(postflood_tiles, desc="Processing post-flood tiles"):
    tile_path = os.path.join(postflood_tile_dir, tile_file)
    compute_ndwi_mask_tile(tile_path, postflood_mask_dir, ndwi_threshold)

print("NDWI masks computed and saved for all tiles!")


Processing post-flood tiles: 100%|██████████| 7038/7038 [05:15<00:00, 22.27it/s]

NDWI masks computed and saved for all tiles!


In [ ]:
"""
Combine 1024*1024 mask tiles back into a full-size raster.
"""
preflood_mask_dir = "/content/masks_preflood"
postflood_mask_dir = "/content/masks_postflood"
preflood_path = "/content/Hanumannagar_Preflood_Orthomosaic.tif"
postflood_path = "/content/Hanumannagar_Postflood_Orthomosaic.tif"


ndwi_threshold = 0.1

def write_tiles_to_full_mask(tile_dir, reference_tif, out_path):
    with rasterio.open(reference_tif) as src:
        profile = src.profile
        profile.update(count=1, dtype=rasterio.uint8)

        with rasterio.open(out_path, "w", **profile) as dst:
            tile_files = os.listdir(tile_dir)
            for tile_file in tqdm(tile_files, desc=f"Writing tiles from {tile_dir}"):
                tile_path = os.path.join(tile_dir, tile_file)

                parts = tile_file.replace(".tif","").split("_")
                i = int(parts[1])
                j = int(parts[2])

                with rasterio.open(tile_path) as t:
                    data = t.read(1)
                    h, w = data.shape
                    window = Window(i, j, w, h)
                    dst.write(data, 1, window=window)

# Write pre-flood and post-flood masks
preflood_full_mask_tif = "preflood_full_mask.tif"
postflood_full_mask_tif = "postflood_full_mask.tif"

write_tiles_to_full_mask(preflood_mask_dir, preflood_path, preflood_full_mask_tif)
write_tiles_to_full_mask(postflood_mask_dir, postflood_path, postflood_full_mask_tif)


Writing tiles from /content/masks_postflood: 100%|██████████| 7038/7038 [01:00<00:00, 116.85it/s]


In [ ]:
"""

Combine the full preflood mask and full postflood mask
✔ Compare them pixel-by-pixel
✔ Detect where water appeared after the flood
✔ Save the result as a new raster:

"""

output_change_mask = "flood_change_mask.tif"
tile_size = 2048  # process in chunks to save RAM

with rasterio.open(preflood_full_mask_tif) as pre, rasterio.open(postflood_full_mask_tif) as post:
    profile = pre.profile
    profile.update(dtype=rasterio.uint8, count=1)

    with rasterio.open(output_change_mask, "w", **profile) as dst:
        for i in range(0, pre.width, tile_size):
            for j in range(0, pre.height, tile_size):
                w = min(tile_size, pre.width - i)
                h = min(tile_size, pre.height - j)
                window = Window(i, j, w, h)

                pre_data = pre.read(1, window=window).astype(int)
                post_data = post.read(1, window=window).astype(int)

                change_data = ((post_data - pre_data) == 1).astype(np.uint8)
                dst.write(change_data, 1, window=window)

print(f"Flood change mask saved as {output_change_mask}")


Flood change mask saved as flood_change_mask.tif


In [ ]:

"""
This is a full flood-polygon extraction workflow: 
converting the flood change mask raster into vector polygons (GeoJSON), tile-by-tile
"""

# -----------------------------
# Inputs
# -----------------------------
change_mask_tif = "flood_change_mask.tif"
output_geojson = "flood_polygons.geojson"

tile_size = 2048  # adjust if RAM is low
temp_dir = "temp_polygons"
os.makedirs(temp_dir, exist_ok=True)

# -----------------------------
# Step 1: Read metadata only
# -----------------------------
with rasterio.open(change_mask_tif) as src:
    width, height = src.width, src.height
    transform = src.transform
    crs = src.crs

# -----------------------------
# Step 2: Process the raster in tiles
# -----------------------------
tile_geojsons = []

with rasterio.open(change_mask_tif) as src:
    for i in tqdm(range(0, width, tile_size), desc="Processing tiles (X-axis)"):
        for j in range(0, height, tile_size):
            w = min(tile_size, width - i)
            h = min(tile_size, height - j)

            window = rasterio.windows.Window(i, j, w, h)
            data = src.read(1, window=window).astype(np.uint8)

            if np.sum(data) == 0:
                continue  # skip empty tiles

            # Compute transform for this tile
            tile_transform = rasterio.windows.transform(window, src.transform)

            # Create polygon features correctly
            results = [
                {
                    "properties": {"value": int(v)},
                    "geometry": shape(s)
                }
                for s, v in features.shapes(data, transform=tile_transform)
                if v == 1
            ]

            if not results:
                continue

            # Save intermediate GeoJSON
            gdf_tile = gpd.GeoDataFrame.from_features(results)
            gdf_tile.set_crs(crs, inplace=True)

            tile_geojson = os.path.join(temp_dir, f"tile_{i}_{j}.geojson")
            gdf_tile.to_file(tile_geojson, driver="GeoJSON")
            tile_geojsons.append(tile_geojson)

# -----------------------------
# Step 3: Merge all tile GeoJSONs
# -----------------------------
print("\nMerging all tile polygons...")
gdfs = [gpd.read_file(fp) for fp in tile_geojsons if os.path.getsize(fp) > 0]
if not gdfs:
    raise ValueError("No flooded polygons were found!")

gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
gdf.set_crs(crs, inplace=True)

# -----------------------------
# Step 4: Compute area and save
# -----------------------------
gdf["area_m2"] = gdf.geometry.area
gdf.to_file(output_geojson, driver="GeoJSON")

print(f"\nFlood polygons saved as {output_geojson}")
print(f"Total polygons: {len(gdf)}")
print(f"Total flooded area: {gdf['area_m2'].sum():,.2f} m²")

# Optional cleanup
# shutil.rmtree(temp_dir)


In [ ]:
import geopandas as gpd

gdf = gpd.read_file("flood_polygons.geojson")
print(gdf.crs)


EPSG:32645


In [ ]:
import geopandas as gpd

# Read your existing polygons
gdf = gpd.read_file("flood_polygons.geojson")

# Convert from UTM (EPSG:32645) to WGS84 (EPSG:4326)
gdf = gdf.to_crs(epsg=4326)

# Save new GeoJSON
gdf.to_file("flood_polygons_wgs84.geojson", driver="GeoJSON")

print("CRS converted to EPSG:4326 and saved as flood_polygons_wgs84.geojson")


In [ ]:
"""
Loads the flood polygons

Centers the map on the flood area

Adds the polygons with blue shading

Saves the result so it can be opened in a web browser
"""
import folium
import geopandas as gpd

geojson_path = "flood_polygons_wgs84.geojson"
gdf = gpd.read_file(geojson_path)

# Center map on flood polygons
m = folium.Map(
    location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()],
    zoom_start=14
)

# Add polygons
folium.GeoJson(
    gdf,
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 1,
        'fillOpacity': 0.5
    }
).add_to(m)

# Auto-zoom to bounds
m.fit_bounds(m.get_bounds())

# Save the map
m.save("flood_map.html")



In [ ]:
# Ensure CRS is defined
print("Current CRS:", gdf.crs)

# Reproject to UTM (EPSG:32645) for accurate area in m²
gdf_utm = gdf.to_crs(epsg=32645)

# Compute area
gdf_utm["area_m2"] = gdf_utm.geometry.area

# Save as GeoJSON
output_geojson = "flood_polygons_metric.geojson"
gdf_utm.to_file(output_geojson, driver="GeoJSON")

print(f"\nFlood polygons saved as {output_geojson}")
print(f"Total polygons: {len(gdf_utm)}")
print(f"Total flooded area: {gdf_utm['area_m2'].sum():,.2f} m²")


In [ ]:
"""
Computes accurate areas in square meters by reprojecting to a suitable UTM coordinate system.

Saves a metric GeoJSON for GIS use.

Converts polygons back to WGS84 (lat/lon) and computes polygon centroids.

Exports a CSV file with polygon centroids, area, and geometry as WKT.

"""

# Inputs

input_geojson = "flood_polygons.geojson"   # from your raster → polygon step
output_geojson_metric = "flood_polygons_metric.geojson"
output_csv = "flood_polygons.csv"


# Load the polygons

gdf = gpd.read_file(input_geojson)
print("Original CRS:", gdf.crs)


# Reproject to UTM (EPSG:32645) for accurate area

gdf_utm = gdf.to_crs(epsg=32645)
gdf_utm["area_m2"] = gdf_utm.geometry.area


# Save accurate version as GeoJSON (in meters)

gdf_utm.to_file(output_geojson_metric, driver="GeoJSON")
print(f"Metric GeoJSON saved: {output_geojson_metric}")


# Convert to WGS84 for lat/lon export

gdf_wgs84 = gdf_utm.to_crs(epsg=4326)

# Compute centroid coordinates for each polygon
gdf_wgs84["centroid_lon"] = gdf_wgs84.geometry.centroid.x
gdf_wgs84["centroid_lat"] = gdf_wgs84.geometry.centroid.y


# Prepare a clean CSV
# Extract polygon coordinates as WKT string
gdf_wgs84["geometry_wkt"] = gdf_wgs84.geometry.to_wkt()

# Select useful columns
csv_df = gdf_wgs84[["centroid_lat", "centroid_lon", "area_m2", "geometry_wkt"]]

# Save to CSV
csv_df.to_csv(output_csv, index=False)

print(f"Flood polygon CSV saved: {output_csv}")
print(f"Total flooded area: {gdf_utm['area_m2'].sum():,.2f} m²")
print(f"Total polygons: {len(gdf_utm)}")



In [ ]:
import pandas as pd

# Input CSV from your script
input_csv = "flood_polygons.csv"
github_handle = "subashsigdel"
output_csv = "flood_regions.csv"

# Read your current CSV
df = pd.read_csv(input_csv)

# Prepare submission-ready CSV
submission_df = pd.DataFrame()
submission_df['tile_id'] = [f"tile_{i:03d}" for i in range(1, len(df)+1)]
submission_df['center_longitude'] = df['centroid_lon']
submission_df['center_latitude'] = df['centroid_lat']
submission_df['area_m2'] = df['area_m2']
submission_df['area_lost_m2'] = 0 
submission_df['pre_flood_land_image'] = [f"submissions/{github_handle}/cutouts/tile_{i:03d}_pre.png" for i in range(1, len(df)+1)]
submission_df['post_flood_land_image'] = [f"submissions/{github_handle}/cutouts/tile_{i:03d}_post.png" for i in range(1, len(df)+1)]

# Save CSV
submission_df.to_csv(output_csv, index=False)
print(f"Submission CSV created: {output_csv}")


In [ ]:
""""
Displays a downsampled post-flood satellite image.

Overlays the flood polygons on top of the image.

"""
# Path to post-flood orthomosaic
tif_path = "/content/Hanumannagar_Postflood_Orthomosaic.tif"

# Downsample factor
factor = 50

with rasterio.open(tif_path) as src:
    # Downsample RGB
    new_height = src.height // factor
    new_width = src.width // factor
    img = src.read([1,2,3], out_shape=(3, new_height, new_width))
    img = reshape_as_image(img)
    img = img.astype(np.uint8)

    # Transform bounds from UTM -> WGS84
    bounds_wgs84 = transform_bounds(src.crs, 'EPSG:4326', *src.bounds)

# Create Folium map centered on raster
m = folium.Map(location=[(bounds_wgs84[1]+bounds_wgs84[3])/2,
                         (bounds_wgs84[0]+bounds_wgs84[2])/2],
               zoom_start=16)

# Overlay raster
folium.raster_layers.ImageOverlay(
    image=img,
    bounds=[[bounds_wgs84[1], bounds_wgs84[0]], [bounds_wgs84[3], bounds_wgs84[2]]],
    opacity=0.6,
    name="Post-flood Image"
).add_to(m)

# Add flood polygons (already in WGS84)
gdf = gpd.read_file("flood_polygons_wgs84.geojson")
folium.GeoJson(
    gdf,
    style_function=lambda x: {'fillColor':'blue','color':'blue','weight':1,'fillOpacity':0.5},
    name="Flood Polygons"
).add_to(m)

folium.LayerControl().add_to(m)
m.save("flood_overlay_map.html")
